In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import sk_feature_selection
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
raw_data = pd.read_csv('../../../data/Base.csv')

In [3]:
raw_data.shape

(1000000, 32)

In [4]:
numeric_vars, str_vars, oth_vars = sort_vars(raw_data, raw_data.columns.tolist())

In [5]:
cap_n_floor = read_yaml_file('../Data_prep/cap_n_floor.yaml')

In [6]:
floors = cap_n_floor['floors']

In [7]:
for floor in floors:
    temp = floor.split(',')
    raw_data[temp[0]] = raw_data[temp[0]].apply(lambda x: np.nan if x < float(temp[1]) else x)

In [8]:
w = pd.ExcelWriter('proc_means.xlsx')
with w as writer:
    proc_means(raw_data, numeric_vars).to_excel(writer,sheet_name = 'all', startcol = 2, startrow = 2)
    proc_means(raw_data.loc[raw_data['fraud_bool']==1], numeric_vars).to_excel(writer,sheet_name = 'fraud=1', startcol = 2, startrow = 2)

In [9]:
raw_data = raw_data.drop(columns=['prev_address_months_count','intended_balcon_amount','bank_months_count']);

In [10]:
raw_data.shape

(1000000, 29)

In [11]:
raw_data = raw_data.dropna()

In [12]:
raw_data.shape

(972571, 29)

In [13]:
numeric_vars.remove('prev_address_months_count')
numeric_vars.remove('intended_balcon_amount')
numeric_vars.remove('bank_months_count')

In [14]:
low_variation_vars =  sk_feature_selection.f_low_variation(raw_data, numeric_vars, 0.05)
low_variation_vars

,feature,rescaled_std,recommendation,outliers
0,email_is_free,0.498973,keep,False
1,keep_alive_session,0.493624,keep,False
2,phone_home_valid,0.493541,keep,False
3,has_other_cards,0.418742,keep,False
4,proposed_credit_limit,0.366070,keep,False
5,income,0.361955,keep,False
6,month,0.315372,keep,False
7,phone_mobile_valid,0.314493,keep,False
8,name_email_similarity,0.290629,keep,False
9,customer_age,0.279296,keep,False


In [15]:
numeric_vars.remove('device_fraud_count')
numeric_vars

['fraud_bool',
 'income',
 'name_email_similarity',
 'current_address_months_count',
 'customer_age',
 'days_since_request',
 'zip_count_4w',
 'velocity_6h',
 'velocity_24h',
 'velocity_4w',
 'bank_branch_count_8w',
 'date_of_birth_distinct_emails_4w',
 'credit_risk_score',
 'email_is_free',
 'phone_home_valid',
 'phone_mobile_valid',
 'has_other_cards',
 'proposed_credit_limit',
 'foreign_request',
 'session_length_in_minutes',
 'keep_alive_session',
 'device_distinct_emails_8w',
 'month']

In [16]:
str_vars

['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']

In [17]:
oth_vars

[]

In [18]:
w = pd.ExcelWriter('proc_means.xlsx')
with w as writer:
    proc_means(raw_data, numeric_vars).to_excel(writer,sheet_name = 'all', startcol = 2, startrow = 2)
    proc_means(raw_data.loc[raw_data['fraud_bool']==1], numeric_vars).to_excel(writer,sheet_name = 'fraud=1', startcol = 2, startrow = 2)

In [19]:
raw_data['orig_weight'] = 1

In [20]:
summary_by_month = pivot(raw_data, varlist={'fraud_bool':('weighted_avg','orig_weight'),'income':'mean', 
                                            'credit_risk_score':'mean'}, by_vars = ['month'])
summary_by_month

,month,count,WA_fraud_bool,mean_income,mean_credit_risk_score
0,0,129626,0.011464,0.5,118.0
1,1,123762,0.009583,0.5,110.0
2,2,133444,0.008865,0.5,112.0
3,3,147725,0.009355,0.6,146.0
4,4,123709,0.011632,0.6,138.0
5,5,116154,0.012044,0.6,143.0
6,6,105305,0.013637,0.6,149.0
7,7,92846,0.015186,0.6,143.0


In [21]:
qcut_vars = ['income','name_email_similarity', 'current_address_months_count', 'customer_age', \
            'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', \
            'date_of_birth_distinct_emails_4w', 'credit_risk_score','proposed_credit_limit', \
            'session_length_in_minutes']

In [22]:
qcut_dict = {}
for var in qcut_vars:
    qcut_dict[var] = [i*0.05 for i in range(1,20)]

In [23]:
raw_data, cps = binning_q(raw_data, qcut_dict)

In [24]:
raw_data.head()

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,payment_type,zip_count_4w,velocity_6h,velocity_24h,...,current_address_months_count_bin,customer_age_bin,velocity_6h_bin,velocity_24h_bin,velocity_4w_bin,bank_branch_count_8w_bin,date_of_birth_distinct_emails_4w_bin,credit_risk_score_bin,proposed_credit_limit_bin,session_length_in_minutes_bin
0,0,0.3,0.986506,25.0,40.0,0.006735,AA,1059,13096.035018,7850.955007,...,"(20.0, 28.0]","(30.0, 40.0]",nan,nan,nan,"(3.0, 6.0]","(4.0, 5.0]","(153.0, 166.0]","(1000.0, 1500.0]","(15.5, 21.6]"
1,0,0.8,0.617426,89.0,20.0,0.010095,AD,1658,9223.283431,5745.251481,...,"(87.0, 108.0]","(19.0, 30.0]","(8776.0, 9631.0]","(5518.0, 5751.0]","(5680.0, 6004.0]","(2.0, 3.0]","(16.0, 19.0]","(153.0, 166.0]","(1000.0, 1500.0]","(3.1, 3.5]"
2,0,0.8,0.996707,14.0,40.0,0.012316,AB,1095,4471.472149,5471.988958,...,"(10.0, 14.0]","(30.0, 40.0]","(4211.0, 4565.0]","(5299.0, 5518.0]","(5680.0, 6004.0]","(13.0, 15.0]","(10.0, 11.0]","(83.0, 92.0]","(199.0, 500.0]",nan
3,0,0.6,0.475100,14.0,30.0,0.006991,AB,3483,14431.993621,6755.344479,...,"(10.0, 14.0]","(19.0, 30.0]",nan,"(6363.0, 6818.0]","(5680.0, 6004.0]","(9.0, 11.0]","(12.0, 13.0]","(83.0, 92.0]","(199.0, 500.0]","(12.3, 15.5]"
4,0,0.9,0.842307,29.0,40.0,5.742626,AA,2339,7601.511579,5124.046930,...,"(28.0, 34.0]","(30.0, 40.0]","(7148.0, 7680.0]","(5102.0, 5299.0]","(5680.0, 6004.0]","(-0.1, 1.0]","(5.0, 6.0]","(83.0, 92.0]","(199.0, 500.0]","(3.5, 3.9]"


In [25]:
figs = []
for var in qcut_vars:
    var_bin = f"{var}_bin"
    xy = pivot(raw_data, varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = [var_bin])
    figs.append(px_scatter_plot(xy, f'mean_{var}', 'logodds_fraud_bool', show=False))

In [26]:
if os.path.exists("logodds_plots.html"):
  os.remove("logodds_plots.html")
    
with open('logodds_plots.html', 'a') as file:
    for f in figs:
        file.write(f.to_html())

In [27]:
raw_data['zip_count_4w_level'] = raw_data['zip_count_4w'].apply(lambda x: 'low' if x <= 950 else ('mid' if 950 < x < 1850 else 'high'))

In [28]:
raw_data['zip_count_4w_level'].value_counts()

zip_count_4w_level
mid     426837
low     279064
high    266670
Name: count, dtype: int64

In [29]:
raw_data['month'].value_counts()

month
3    147725
2    133444
0    129626
1    123762
4    123709
5    116154
6    105305
7     92846
Name: count, dtype: int64

In [30]:
figs = []
for var in qcut_vars:
    var_bin = f"{var}_bin"

    xy1 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='low'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])
    xy2 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='mid'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])
    xy3 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='high'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])

    xy1['zip_count_4w_level'] = 'low'
    xy2['zip_count_4w_level'] = 'mid'
    xy3['zip_count_4w_level'] = 'high'
    
    xy = pd.concat([xy1,xy2,xy3],axis=0)

    figs.append(px_scatter_plot(xy, f'mean_{var}', 'logodds_fraud_bool', by_var1 = 'month', by_var2 = 'zip_count_4w_level' ,show=False))


In [31]:
if os.path.exists("logodds_plots_level.html"):
  os.remove("logodds_plots_level.html")

In [32]:
with open('logodds_plots_level.html', 'a') as file:
    for f in figs:
        file.write(f.to_html())

In [33]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','month')
col_pct

month,0,1,2,3,4,5,6,7
fraud_bool,,,,,,,,
0,0.133250,0.127464,0.137535,0.152179,0.127146,0.119331,0.108011,0.095082
1,0.136068,0.108598,0.108323,0.126545,0.131764,0.128102,0.131490,0.129109


In [34]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','device_os')
col_pct

device_os,linux,macintosh,other,windows,x11
fraud_bool,,,,,
0,0.337781,0.052200,0.341984,0.260787,0.007248
1,0.155755,0.068217,0.177365,0.591338,0.007325


In [35]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','payment_type')
col_pct

payment_type,AA,AB,AC,AD,AE
fraud_bool,,,,,
0,0.260136,0.372337,0.248752,0.118485,0.000290
1,0.124073,0.378811,0.380643,0.116381,0.000092


In [36]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','housing_status')
col_pct

housing_status,BA,BB,BC,BD,BE,BF,BG
fraud_bool,,,,,,,
0,0.168285,0.266587,0.375845,0.026468,0.160881,0.001677,0.000258
1,0.577969,0.143302,0.207215,0.020603,0.050179,0.000641,0.000092


In [37]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','device_os')
col_pct

device_os,linux,macintosh,other,windows,x11
fraud_bool,,,,,
0,0.337781,0.052200,0.341984,0.260787,0.007248
1,0.155755,0.068217,0.177365,0.591338,0.007325


In [38]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','source')
col_pct

source,INTERNET,TELEAPP
fraud_bool,,
0,0.994616,0.005384
1,0.991393,0.008607


In [39]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','housing_status')
col_pct

housing_status,BA,BB,BC,BD,BE,BF,BG
fraud_bool,,,,,,,
0,0.168285,0.266587,0.375845,0.026468,0.160881,0.001677,0.000258
1,0.577969,0.143302,0.207215,0.020603,0.050179,0.000641,0.000092


In [40]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','payment_type')
col_pct

payment_type,AA,AB,AC,AD,AE
fraud_bool,,,,,
0,0.260136,0.372337,0.248752,0.118485,0.000290
1,0.124073,0.378811,0.380643,0.116381,0.000092


In [41]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','employment_status')

In [42]:
col_pct

employment_status,CA,CB,CC,CD,CE,CF,CG
fraud_bool,,,,,,,
0,0.727028,0.140980,0.037957,0.026927,0.021362,0.045302,0.000445
1,0.805695,0.086988,0.085065,0.009157,0.004853,0.007692,0.000549


In [43]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','phone_home_valid')
col_pct

phone_home_valid,0,1
fraud_bool,,
0,0.578222,0.421778
1,0.746269,0.253731


In [44]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','phone_mobile_valid')
col_pct

phone_mobile_valid,0,1
fraud_bool,,
0,0.110852,0.889148
1,0.149986,0.850014


In [45]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','has_other_cards')
col_pct

has_other_cards,0,1
fraud_bool,,
0,0.771623,0.228377
1,0.914935,0.085065


In [46]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','foreign_request')
col_pct

foreign_request,0,1
fraud_bool,,
0,0.974795,0.025205
1,0.949455,0.050545


In [47]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','source')
col_pct

source,INTERNET,TELEAPP
fraud_bool,,
0,0.994616,0.005384
1,0.991393,0.008607


In [48]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','keep_alive_session')
col_pct

keep_alive_session,0,1
fraud_bool,,
0,0.417714,0.582286
1,0.657449,0.342551


In [49]:
ttest = grp_ttest(qcut_vars,raw_data.loc[raw_data['fraud_bool']==0], raw_data.loc[raw_data['fraud_bool']==1])

In [50]:
ttest

,income,name_email_similarity,current_address_months_count,customer_age,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,proposed_credit_limit,session_length_in_minutes
T-statistic,-44.303854,36.02909,-34.030301,-62.087321,16.878267,11.123916,11.734926,11.521271,43.373489,-69.98756,-68.32531,-9.000193
P-value,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
selector = sk_feature_selection()

In [52]:
cont_vars = ['income','name_email_similarity', 'current_address_months_count', 'customer_age', \
             'zip_count_4w', 'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', \
            'date_of_birth_distinct_emails_4w', 'credit_risk_score','proposed_credit_limit', \
            'session_length_in_minutes','month']

In [53]:
disc_vars = ['email_is_free','phone_home_valid','phone_mobile_valid','has_other_cards','foreign_request','keep_alive_session','device_distinct_emails_8w']

In [54]:
mi = selector.f_normalized_mi_matrix(raw_data, ['fraud_bool']+cont_vars+disc_vars)

In [55]:
mi.to_excel('nmi_matrix.xlsx')

In [56]:
coef = selector.f_feature_select(raw_data, cont_vars + disc_vars, 'fraud_bool', mtype='classification', chart='off')

In [57]:
coef.to_excel('feature_importance.xlsx')

In [58]:
raw_data['fraud_bool'].value_counts()

fraud_bool
0    961650
1     10921
Name: count, dtype: int64

In [59]:
raw_data_dummy = f_get_dummies(raw_data, varlist=['payment_type','employment_status','housing_status','source','device_os','month'], drop_first=False)

In [60]:
dummy_features = [x for x in raw_data_dummy.columns if x not in raw_data.columns]

In [61]:
dummy_features

['payment_type:AA',
 'payment_type:AB',
 'payment_type:AC',
 'payment_type:AD',
 'payment_type:AE',
 'payment_type:nan',
 'employment_status:CA',
 'employment_status:CB',
 'employment_status:CC',
 'employment_status:CD',
 'employment_status:CE',
 'employment_status:CF',
 'employment_status:CG',
 'employment_status:nan',
 'housing_status:BA',
 'housing_status:BB',
 'housing_status:BC',
 'housing_status:BD',
 'housing_status:BE',
 'housing_status:BF',
 'housing_status:BG',
 'housing_status:nan',
 'source:INTERNET',
 'source:TELEAPP',
 'source:nan',
 'device_os:linux',
 'device_os:macintosh',
 'device_os:other',
 'device_os:windows',
 'device_os:x11',
 'device_os:nan',
 'month:0.0',
 'month:1.0',
 'month:2.0',
 'month:3.0',
 'month:4.0',
 'month:5.0',
 'month:6.0',
 'month:7.0',
 'month:nan']

In [62]:
coef = selector.f_feature_select(raw_data_dummy, dummy_features, 'fraud_bool', mtype='classification', chart='off')

In [63]:
coef.to_excel('dummy_feature_importance.xlsx')